### Load data from GPKG

In [1]:
import geopandas
import googlemaps
import pandas as pd
import numpy as np
import credentials  # Here is my google api key... you need to put you own in this file.

trips=geopandas.read_file("data/Ud_I_Naturen.gpkg", layer='lines')
trips["wgs84_geometry"] = trips["geometry"].to_crs(epsg=4326)

### Functions

In [2]:
# Converts a list of gps points to a dataframe with elevation data
def getGoogleElevations(tripPoints, tripLength, wantedPoints):
    tripPoints = [(point[1], point[0]) for point in tripPoints]  # Google want lat/lon exhanged in the list
    gmaps = googlemaps.Client(key=credentials.googleApiKey)

    elevationPoints = gmaps.elevation_along_path(tripPoints, wantedPoints)

    elevationPanda = pd.DataFrame(elevationPoints)
    locations = pd.DataFrame(elevationPanda.location.to_list())
    elevationPanda = elevationPanda.join(locations)
    elevationPanda = elevationPanda.drop(["location", "resolution"], axis=1)

    interpolatedMeters = np.arange(0, tripLength, tripLength/wantedPoints)
    elevationPanda["meters"] = interpolatedMeters[:wantedPoints] #rounding sometimes makes one value too much
    return elevationPanda

### Traverse all line data in gpkg and save elevation data for each trip

In [3]:
for idx, trip in trips.iterrows():
    tripLength = trip.geometry.length
    tripId = trip.facilityId
    tripCoordinates = trip.wgs84_geometry.coords
    try:
        googleElevations = getGoogleElevations(tripCoordinates, tripLength, 500)
        googleElevations.to_csv(f"data/elevation_{tripId}.csv")  # Save elevation data to csv file
    except:
        print(f"Trip={tripId}, Length={tripLength}m, FAIL!!!!!!")

Trip=15715, Length=128893.90826165183m, FAIL!!!!!!
Trip=9150, Length=326677.15904793073m, FAIL!!!!!!
Trip=21782, Length=0.0m, FAIL!!!!!!
Trip=18907, Length=0.0m, FAIL!!!!!!
Trip=16993, Length=0.0m, FAIL!!!!!!
Trip=22141, Length=0.0m, FAIL!!!!!!
Trip=9277, Length=20967.02367993733m, FAIL!!!!!!
Trip=22713, Length=19756.31651495097m, FAIL!!!!!!
Trip=15230, Length=0.0m, FAIL!!!!!!
Trip=18354, Length=0.0m, FAIL!!!!!!
Trip=19838, Length=96744.95588350573m, FAIL!!!!!!
Trip=19841, Length=65304.889449106966m, FAIL!!!!!!
Trip=19833, Length=55514.3644483646m, FAIL!!!!!!
Trip=20306, Length=0.0m, FAIL!!!!!!
Trip=22814, Length=380332.1179628715m, FAIL!!!!!!
Trip=23051, Length=661828.7643295493m, FAIL!!!!!!
Trip=19843, Length=72624.80933609407m, FAIL!!!!!!
Trip=20540, Length=450515.53367961716m, FAIL!!!!!!
Trip=14421, Length=0.0m, FAIL!!!!!!
Trip=16302, Length=0.0m, FAIL!!!!!!
Trip=16349, Length=0.0m, FAIL!!!!!!
Trip=15198, Length=0.0m, FAIL!!!!!!
Trip=17161, Length=0.0m, FAIL!!!!!!
Trip=19685, Lengt